In [1]:
import pandas as pd
import json
import numpy as np
import re
import numpy as np
from dateutil.parser import parse as dateparse
from lat_lon_parser import parse as parse
import json
import re
import reverse_geocode as rg

############## Accès S3 quand ça marchera ############
#from pyarrow import fs
#
#s3fs = fs.S3FileSystem(
#    endpoint_override="localhost:9000",
#    access_key="lbIwWP4FIBtjBwU5AaI0",
#    secret_key="nw54mjkG3CjxjvkyqxACTbYOuhtzyc1YmkMaJXeL",
#    scheme="http"
#)
#
#s3fs.get_file_info('geo-sra/sra')
######################################################

df = pd.read_parquet("/Users/tpietav/Desktop/data/raw/sra_metadata_toy.parquet")

In [2]:
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        dateparse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [3]:
def select_first_collection_date(x):
    if not pd.isnull(x):
        return x[0]
    else:
        return None
    
df["collection_date"] = df['collection_date_sam'].apply(select_first_collection_date)

In [4]:
def is_alpha_string(string,max_words=2):
    """
    Return wether a string is an alpha string or not

    Args:
        string (String): string to check 
        max_words (int, optional): max words that can contain the string. Defaults to 2.
    """
    for n in range(max_words):
        pattern = r'^[a-zA-Z]+ {'+str(n)+r'}[a-zA-Z]+$'
        if re.match(pattern,string):
            return True
    
    return False

In [5]:
def decimal_precision(nombre):
    if nombre == np.NaN:
        return np.NaN
    chaine = str(nombre)
    index_point = chaine.find('.')
    if index_point == -1:
        return np.NaN
    return len(chaine) - index_point - 1

In [25]:
def country_is_na(country):
    """
    Return wether a country field is NA or not

    Args:
        country (string): Country value

    Returns:
        bool: True if the string is Na False if not
    """
    country_na_tag = ['uncalculated','<NA>']
    if pd.isna(country):
        return True
    elif country in country_na_tag:
        return True
    return False

In [7]:
import multiprocessing

num_partitions = multiprocessing.cpu_count()

print('#START - Parse data')
print('num_partitions',num_partitions,'----')

#START - Parse data
num_partitions 11 ----


In [8]:
import dask.dataframe as ddf

dask = ddf.from_pandas(df, npartitions=num_partitions-5)

In [17]:
import datetime
columns = ['bioproject', 'acc','organism','assay_type','instrument','librarylayout','libraryselection','librarysource','geo_loc_name_country_calc','geo_loc_name_country_continent_calc','mbytes','mbases','releasedate', 'collection_date','lat_lon_src','lat_lon_raw','latitude','longitude','latitude_precision','has_latlon','longitude_precision','rg_country_code','rg_city','rg_country','GEO_QUAL']
dtype = [str,str,str,str,str,str,str,str,str,str,int,int,datetime.date,datetime.date,str,str,float,float,float,float,bool,str,str,str,int]
cdt={i[0]: i[1] for i in zip(columns, dtype)}
meta_df = pd.DataFrame(columns=list(cdt))

In [10]:
# TAGS TEST

dic = {
    "Lat Lon Tag":0,
    "Lat Tag"    :0,
    "Lon Tag"    :0,
    "No Tag"     :0
}


###########################################################################
# 
# selected tag
#

lat_lon_tag = ['lat_lon_sam_s_dpl34','lat_lon_sam_s_dpl1','geographic_location__latitude_and_longitude__sam','geographic_location__latitudeandlongitude__sam','latitude__and_longitude_sam','latitude_and_longitude_sam','lattitude_and_logitude_sam','lat_lon_sam','latlon_sam','location_coordinates_sam','other_gps_coordinates_sam','lat_lon_dms_sam','lat_long_correct_sam','lat_lon_run']

lat_tag = ['geographic_location__latitude__sam_s_dpl4','lat_lon_sam_s_dpl1','latitude_sam','lat_sam','geographic_location__latitude__sam','geographiclocation_latitude__sam','latitude_dd_sam','latitude_deg_sam','biological_material_latitude_sam']

lon_tag = ['geographic_location__longitude__sam_s_dpl5','longitude_sam','lon_sam','geographic_location__longitude__sam','longitude_dd_sam','longitude_deg_sam','biological_material_longitude_sam']
        
filtered_tag = np.concatenate((lat_lon_tag,lat_tag,lon_tag))

for samples in dask.itertuples(index=False):
     
    # Problem of serialization between dask partition / pandas
    # attributes = samples.attributes
    # attributes_df = pd.DataFrame.from_records(attributes)

    attributes = samples.jattr

    if(not attributes==""):

        attributes_df = pd.DataFrame(json.loads(attributes).items(),columns=['k', 'v'])

    else:
        
        attributes_df=pd.DataFrame()
        print("ERROR EMPTY")

    if('k' in attributes_df):
            
            attributes_df = attributes_df.loc[attributes_df['k'].isin(filtered_tag)]

            for index, attribute in attributes_df.iterrows():
                
                #store attributes keyword

                #if attribute['k'] in dict_attributes_k :
                #    dict_attributes_k[attribute['k']] = dict_attributes_k[attribute['k']] + 1
                #else :
                #    dict_attributes_k[attribute['k']] = 0
                #    print("keyword", attribute['k'])

                ###########################################################################
                # latitude - longitude in the same field

                lat_lon_list = None

                if attribute['k'] in lat_lon_tag : 
                    dic["Lat Lon Tag"] += 1
                if attribute['k'] in lat_tag:
                    dic['Lat Tag'] += 1
                if attribute['k'] in lon_tag:
                    dic['Lon Tag'] += 1
                if not ((attribute['k'] in lat_lon_tag) or (attribute['k'] in lat_tag) or (attribute['k'] in lon_tag)):
                    dic['No Tag'] += 1

dic

{'Lat Lon Tag': 280107, 'Lat Tag': 22640, 'Lon Tag': 22410, 'No Tag': 0}

In [11]:
# LAT LON PATTERNS TEST

lat_lon_patterns = [
    r"(-?\d+\.\d+ [NS]) (-?\d+\.\d+ [WE])",   # xx.xxx N xx.xxx W
    r"(-?\d+\.\d+' [NS]) (-?\d+\.\d+' [WE])", # xx.xxx' N xx.xxx' W
    r"(-?\d+\.\d+° [NS]) (-?\d+\.\d+° [WE])", # xx.xxx° N xx.xxx° W
    r'(-?\d+ [NS]) (-?\d+ [WE])',             # xx N xx W
    r'(-?\d+\.\d+ [NS]) (-?\d+ [WE])',        # xx.xxx N xx W
    r'(-?\d+ [NS]) (-?\d+\.\d+ [WE])',        # xx N xx.xxx W 
    r'(-?\d+\.\d+) (-?\d+\.\d+)',             # xx.xxx xx.xxx
    r'(-?\d+\.\d+°) (-?\d+\.\d+°)',           # xx.xxx° xx.xxx°
    r"(-?\d+\.\d+') (-?\d+\.\d+')",           # xx.xxx' xx.xxx'
]

dic = {
    "Match"       :0,
    "Alpha String":0,
    "No Match"   :0,
    "Tot"         :0
}


###########################################################################
# 
# selected tag
#

lat_lon_tag = ['lat_lon_sam_s_dpl34','lat_lon_sam_s_dpl1','geographic_location__latitude_and_longitude__sam','geographic_location__latitudeandlongitude__sam','latitude__and_longitude_sam','latitude_and_longitude_sam','lattitude_and_logitude_sam','lat_lon_sam','latlon_sam','location_coordinates_sam','other_gps_coordinates_sam','lat_lon_dms_sam','lat_long_correct_sam','lat_lon_run']

lat_tag = ['geographic_location__latitude__sam_s_dpl4','lat_lon_sam_s_dpl1','latitude_sam','lat_sam','geographic_location__latitude__sam','geographiclocation_latitude__sam','latitude_dd_sam','latitude_deg_sam','biological_material_latitude_sam']

lon_tag = ['geographic_location__longitude__sam_s_dpl5','longitude_sam','lon_sam','geographic_location__longitude__sam','longitude_dd_sam','longitude_deg_sam','biological_material_longitude_sam']
        
filtered_tag = np.concatenate((lat_lon_tag,lat_tag,lon_tag))

for samples in dask.itertuples(index=False):
     
    # Problem of serialization between dask partition / pandas
    # attributes = samples.attributes
    # attributes_df = pd.DataFrame.from_records(attributes)

    attributes = samples.jattr

    if(not attributes==""):

        attributes_df = pd.DataFrame(json.loads(attributes).items(),columns=['k', 'v'])

    else:
        
        attributes_df=pd.DataFrame()
        print("ERROR EMPTY")

    if('k' in attributes_df):
            
            attributes_df = attributes_df.loc[attributes_df['k'].isin(filtered_tag)]

            for index, attribute in attributes_df.iterrows():
                
                #store attributes keyword

                #if attribute['k'] in dict_attributes_k :
                #    dict_attributes_k[attribute['k']] = dict_attributes_k[attribute['k']] + 1
                #else :
                #    dict_attributes_k[attribute['k']] = 0
                #    print("keyword", attribute['k'])

                ###########################################################################
                # latitude - longitude in the same field

                lat_lon_list = None

                if attribute['k'] in lat_lon_tag :

                    dic['Tot'] += 1
                    
                    lat_lon_src = attribute['k']

                    if(isinstance(attribute['v'], list)):
                        lat_lon = attribute['v'][0]
                    else:
                        lat_lon = attribute['v']

                    if not is_alpha_string(str(lat_lon)):
                        str_lat_lon = str(lat_lon)
                        for pattern in lat_lon_patterns:
                            if re.match(pattern,str_lat_lon):
                                dic['Match'] += 1
                                lat_lon_list = re.match(pattern,str_lat_lon)
                        if lat_lon_list is None:
                            dic['No Match'] += 1
                    else:
                        dic['Alpha String'] += 1

dic

{'Match': 239553, 'Alpha String': 39424, 'No Match': 1130, 'Tot': 280107}

In [12]:
# LAT & LON PATTERNS TEST

lat_patterns = [
    r"(-?\d+\.\d+ [NS])",   # xx.xxx N xx.xxx W
    r"(-?\d+\.\d+' [NS])",  # xx.xxx' N xx.xxx' W
    r"(-?\d+\.\d+° [NS])",  # xx.xxx° N xx.xxx° W
    r'(-?\d+ [NS])',             # xx N xx W
    r'(-?\d+\.\d+ [NS])',        # xx.xxx N xx W
    r'(-?\d+\.\d+)',             # xx.xxx xx.xxx
    r'(-?\d+\.\d+°)',           # xx.xxx° xx.xxx°
    r"(-?\d+\.\d+')",           # xx.xxx' xx.xxx'
]

lon_patterns = [
    r"(-?\d+\.\d+ [WE])",
    r"(-?\d+\.\d+' [WE])",
    r"(-?\d+\.\d+° [WE])",
    r"(-?\d+ [WE])",
    r"(-?\d+\.\d+ [WE])",
    r"(-?\d+\.\d+)",
    r"(-?\d+\.\d+°)",
    r"(-?\d+\.\d+')"
]

dic = {
    "Lon & Lat Match":0,
    "Lon Match"      :0,
    "Lat Match"      :0,
    "No Match"       :0,
    "Incomplete"     :0
}


###########################################################################
# 
# selected tag
#

lat_lon_tag = ['lat_lon_sam_s_dpl34','lat_lon_sam_s_dpl1','geographic_location__latitude_and_longitude__sam','geographic_location__latitudeandlongitude__sam','latitude__and_longitude_sam','latitude_and_longitude_sam','lattitude_and_logitude_sam','lat_lon_sam','latlon_sam','location_coordinates_sam','other_gps_coordinates_sam','lat_lon_dms_sam','lat_long_correct_sam','lat_lon_run']

lat_tag = ['geographic_location__latitude__sam_s_dpl4','lat_lon_sam_s_dpl1','latitude_sam','lat_sam','geographic_location__latitude__sam','geographiclocation_latitude__sam','latitude_dd_sam','latitude_deg_sam','biological_material_latitude_sam']

lon_tag = ['geographic_location__longitude__sam_s_dpl5','longitude_sam','lon_sam','geographic_location__longitude__sam','longitude_dd_sam','longitude_deg_sam','biological_material_longitude_sam']
        
filtered_tag = np.concatenate((lat_lon_tag,lat_tag,lon_tag))

for samples in dask.itertuples(index=False):
     
    # Problem of serialization between dask partition / pandas
    # attributes = samples.attributes
    # attributes_df = pd.DataFrame.from_records(attributes)

    attributes = samples.jattr
    match_score = 0

    if(not attributes==""):

        attributes_df = pd.DataFrame(json.loads(attributes).items(),columns=['k', 'v'])

    else:
        
        attributes_df=pd.DataFrame()
        print("ERROR EMPTY")

    if('k' in attributes_df):
            
            attributes_df = attributes_df.loc[attributes_df['k'].isin(filtered_tag)]

            for index, attribute in attributes_df.iterrows():

                if attribute['k'] in lat_tag :
                    match_score += 0.5
                    if(isinstance(attribute['v'], list)):
                        lat = attribute['v'][0]
                    else:
                        lat = attribute['v']

                    if not is_alpha_string(str(lat)):
                        str_lat = str(lat)
                        for pattern in lat_patterns:
                            if re.match(pattern,str_lat):
                                match_score += 1
                            
                if attribute['k'] in lon_tag :
                    match_score += 0.5
                    if(isinstance(attribute['v'], list)):
                        lon = attribute['v'][0]
                    else:
                        lon = attribute['v']

                    if not is_alpha_string(str(lon)):
                        str_lon = str(lon)
                        for pattern in lon_patterns:
                            if re.match(pattern,str_lon):
                                match_score += 2
    
    if np.floor(match_score) != match_score :
        dic['Incomplete'] += 1
    elif match_score == 1:
        dic['No Match'] += 1
    elif match_score == 2:
        dic['Lat Match'] += 1
    elif match_score == 3:
        dic['Lon Match'] += 1
    elif match_score == 4:
        dic['Lon & Lat Match'] += 1

dic

{'Lon & Lat Match': 19234,
 'Lon Match': 98,
 'Lat Match': 372,
 'No Match': 2622,
 'Incomplete': 230}

In [26]:
def parse_geo_sra(pd_array_partition): 

    array_result = []

    for samples in pd_array_partition.itertuples(index=False):

        # initialise variable

        lat_lon_src = ''
        lat_lon_raw = ''
        lat_src = ''
        lat_raw = ''

        lon_src = ''
        lon_raw = ''
        
        lat_lon = ''
        lat = ''
        lon = ''
        lat_list = None
        lon_list = None
        lat_lon_list = None

        latitude = None
        longitude = None

        latitude_precision = None
        longitude_precision = None

        has_latlon = False

        geo_qual = 4

        #
        # parse from metadata available
        #

        bioproject = samples.bioproject
        acc = samples.acc
        organism = samples.organism
        assay_type = samples.assay_type
        instrument = samples.instrument
        librarylayout = samples.librarylayout
        libraryselection = samples.libraryselection
        librarysource = samples.librarysource
        geo_loc_name_country_calc = samples.geo_loc_name_country_calc
        geo_loc_name_country_continent_calc = samples.geo_loc_name_country_continent_calc
        mbytes = samples.mbytes
        mbases = samples.mbases

        releasedate = None
        if( not pd.isnull(samples.releasedate)) :
            if(is_date(samples.releasedate)):
                 releasedate = pd.to_datetime(samples.releasedate)

        collection_date = None
        if( not pd.isnull(samples.collection_date)) :
            # multiple collection date are sometimes available for the same sample
            # take the first one
            if(is_date(samples.collection_date)):
                collection_date = pd.to_datetime(samples.collection_date)

        #print("DATE",acc,collection_date,releasedate)

        # 
        # reverse geocoding
        #

        rg_country_code = ''
        rg_city = ''
        rg_country = ''

        #
        # parse from attributes metadata available
        #

        lat_lon_patterns = [
            r"(-?\d+\.\d+ [NS]) (-?\d+\.\d+ [WE])",   # xx.xxx N xx.xxx W
            r"(-?\d+\.\d+' [NS]) (-?\d+\.\d+' [WE])", # xx.xxx' N xx.xxx' W
            r"(-?\d+\.\d+° [NS]) (-?\d+\.\d+° [WE])", # xx.xxx° N xx.xxx° W
            r'(-?\d+ [NS]) (-?\d+ [WE])',             # xx N xx W
            r'(-?\d+\.\d+ [NS]) (-?\d+ [WE])',        # xx.xxx N xx W
            r'(-?\d+ [NS]) (-?\d+\.\d+ [WE])',        # xx N xx.xxx W 
            r'(-?\d+\.\d+) (-?\d+\.\d+)',             # xx.xxx xx.xxx
            r'(-?\d+\.\d+°) (-?\d+\.\d+°)',           # xx.xxx° xx.xxx°
            r"(-?\d+\.\d+') (-?\d+\.\d+')",           # xx.xxx' xx.xxx'
        ]

        lat_patterns = [
            r"(-?\d+\.\d+ [NS])",   # xx.xxx N xx.xxx W
            r"(-?\d+\.\d+' [NS])",  # xx.xxx' N xx.xxx' W
            r"(-?\d+\.\d+° [NS])",  # xx.xxx° N xx.xxx° W
            r'(-?\d+ [NS])',             # xx N xx W
            r'(-?\d+\.\d+ [NS])',        # xx.xxx N xx W
            r'(-?\d+\.\d+)',             # xx.xxx xx.xxx
            r'(-?\d+\.\d+°)',           # xx.xxx° xx.xxx°
            r"(-?\d+\.\d+')",           # xx.xxx' xx.xxx'
        ]

        lon_patterns = [
            r"(-?\d+\.\d+ [WE])",
            r"(-?\d+\.\d+' [WE])",
            r"(-?\d+\.\d+° [WE])",
            r"(-?\d+ [WE])",
            r"(-?\d+\.\d+ [WE])",
            r"(-?\d+\.\d+)",
            r"(-?\d+\.\d+°)",
            r"(-?\d+\.\d+')"
        ]

        ###########################################################################
        # 
        # selected tag
        #

        lat_lon_tag = ['lat_lon_sam_s_dpl34','lat_lon_sam_s_dpl1','geographic_location__latitude_and_longitude__sam','geographic_location__latitudeandlongitude__sam','latitude__and_longitude_sam','latitude_and_longitude_sam','lattitude_and_logitude_sam','lat_lon_sam','latlon_sam','location_coordinates_sam','other_gps_coordinates_sam','lat_lon_dms_sam','lat_long_correct_sam','lat_lon_run']

        lat_tag = ['geographic_location__latitude__sam_s_dpl4','lat_lon_sam_s_dpl1','latitude_sam','lat_sam','geographic_location__latitude__sam','geographiclocation_latitude__sam','latitude_dd_sam','latitude_deg_sam','biological_material_latitude_sam']

        lon_tag = ['geographic_location__longitude__sam_s_dpl5','longitude_sam','lon_sam','geographic_location__longitude__sam','longitude_dd_sam','longitude_deg_sam','biological_material_longitude_sam']
        
        filtered_tag = np.concatenate((lat_lon_tag,lat_tag,lon_tag))

        # Problem of serialization between dask partition / pandas
        # attributes = samples.attributes
        # attributes_df = pd.DataFrame.from_records(attributes)


        attributes = samples.jattr

        if(not attributes==""):

            attributes_df = pd.DataFrame(json.loads(attributes).items(),columns=['k', 'v'])

        else:
            attributes_df=pd.DataFrame()
            print("ERROR EMPTY")

        ###########################################################################
        
        if('k' in attributes_df):
            
            attributes_df = attributes_df.loc[attributes_df['k'].isin(filtered_tag)]

            for index, attribute in attributes_df.iterrows():
                
                #store attributes keyword

                #if attribute['k'] in dict_attributes_k :
                #    dict_attributes_k[attribute['k']] = dict_attributes_k[attribute['k']] + 1
                #else :
                #    dict_attributes_k[attribute['k']] = 0
                #    print("keyword", attribute['k'])

                ###########################################################################
                # latitude - longitude in the same field


                if attribute['k'] in lat_lon_tag :
                    
                    lat_lon_src = attribute['k']

                    if(isinstance(attribute['v'], list)):
                        lat_lon = attribute['v'][0]
                        lat_lon_raw = attribute['v'][0]
                    else:
                        lat_lon = attribute['v']
                        lat_lon_raw = attribute['v']

                    if not is_alpha_string(str(lat_lon)):
                        for pattern in lat_lon_patterns:
                            if re.match(pattern,str(lat_lon)):
                                lat_lon_list = re.match(pattern,str(lat_lon))

                ###########################################################################

                ###########################################################################
                # latitude - longitude in separated fields

                # latitude

                if attribute['k'] in lat_tag :
                    
                    lat_src = attribute['k']

                    if(isinstance(attribute['v'], list)):
                        lat = attribute['v'][0]
                        lat_raw = attribute['v'][0]
                    else:
                        lat = attribute['v']
                        lat_raw = attribute['v']

                    if lat_lon_src == '':
                        lat_lon_src = lat_src
                        lat_lon_raw = lat_raw
                    else:
                        lat_lon_src = lat_src + " " + lat_lon_src
                        lat_lon_raw = str(lat_raw) + " " + lat_lon_raw

                    if not is_alpha_string(str(lat)) :
                        for pattern in lat_patterns:
                            if re.match(pattern,str(lat)):
                                lat_list = re.match(pattern,str(lat))

                # longitude
                
                if attribute['k'] in lon_tag :
                    
                    lon_src = attribute['k']
                    
                    if(isinstance(attribute['v'], list)):
                        lon = attribute['v'][0]
                        lon_raw = attribute['v'][0]
                    else:
                        lon = attribute['v']
                        lon_raw = attribute['v']
                    
                    if lat_lon_src == '':
                        lat_lon_src = lon_src
                        lat_lon_raw = lon_raw
                    else:
                        lat_lon_src = lat_lon_src + " " + lon_src
                        lat_lon_raw = lat_lon_raw + " " + str(lon_raw)

                    if not is_alpha_string(str(lon)) :
                        for pattern in lon_patterns:
                            if re.match(pattern,str(lon)):
                                lon_list = re.match(pattern,str(lon))
                
                ###########################################################################

            ###########################################################################
                
            #
            # lat_lon_parser
            #

            if lat_lon_list is not None :

                try :
                    latitude = parse(lat_lon_list.group(1))
                    longitude = parse(lat_lon_list.group(2))
                    latitude_precision = decimal_precision(latitude)
                    longitude_precision = decimal_precision(longitude)
                except:
                    #print("parse ERROR", acc,lat_lon)
                    pass

            if (lon_list is not None) & (lat_list is not None) & (str(lon).count(".") <=1) & (str(lat).count(".") <=1) :

                if (lon_list.group(1) is not None) & (lat_list.group(1) is not None) :

                    try :
                        latitude = parse(lat_list.group(1))
                        longitude = parse(lon_list.group(1))
                        latitude_precision = decimal_precision(latitude)
                        longitude_precision = decimal_precision(longitude)
                    except :
                        #print("parse ERROR", acc,lat, lon)
                        pass

            #
            # reverse geocoding
            #
            
            if((latitude is not None) & (longitude is not None)):
                coordinates = [(latitude,longitude)]
                has_latlon=True
                rgeocode = rg.search(coordinates)
                
                if(len(rgeocode)>0):
                    rg_country_code = rgeocode[0]['country_code']
                    rg_city = rgeocode[0]['city']
                    rg_country = rgeocode[0]['country']
            
            #
            # GEO_QUAL calc
            #
            
            thr_prec = 2
            if ((latitude is not None) & (longitude is not None)):
                if ((latitude_precision > thr_prec) | (longitude_precision > thr_prec)):
                    geo_qual = 1
                else:
                    geo_qual = 2
            elif (((latitude is None) | (longitude is None)) & (not country_is_na(geo_loc_name_country_calc))):
                geo_qual = 3
            
        # append result
        array_result.append([bioproject,acc,organism,assay_type,instrument,librarylayout,libraryselection,librarysource,geo_loc_name_country_calc,geo_loc_name_country_continent_calc,mbytes,mbases,releasedate,collection_date,lat_lon_src,lat_lon_raw,latitude,longitude,latitude_precision,has_latlon,longitude_precision,rg_country_code,rg_city,rg_country,geo_qual])

    columns = ['bioproject', 'acc','organism','assay_type','instrument','librarylayout','libraryselection','librarysource','geo_loc_name_country_calc','geo_loc_name_country_continent_calc','mbytes','mbases','releasedate', 'collection_date','lat_lon_src','lat_lon_raw','latitude','longitude','latitude_precision','has_latlon','longitude_precision','rg_country_code','rg_city','rg_country','GEO_QUAL']
    dtype = [str,str,str,str,str,str,str,str,str,str,int,int,datetime.date,datetime.date,str,str,float,float,float,float,bool,str,str,str,int]
    cdt={i[0]: i[1] for i in zip(columns, dtype)}        
    pd_result = pd.DataFrame(data=array_result,columns=list(cdt))

    return pd_result

In [27]:
pd_output = dask.map_partitions(parse_geo_sra, meta=meta_df).compute(scheduler='multiprocessing')
pd_output[["bioproject","latitude","latitude_precision","longitude","longitude_precision","geo_loc_name_country_calc","GEO_QUAL"]]

,bioproject,latitude,latitude_precision,longitude,longitude_precision,geo_loc_name_country_calc,GEO_QUAL
0,PRJNA701021,55.226417,6.0,-77.695722,6.0,Canada,1
1,PRJNA291529,61.130000,2.0,25.240000,2.0,Finland,2
2,PRJNA780327,40.488160,5.0,-74.426730,5.0,USA,1
3,PRJNA701021,55.226528,6.0,-77.696917,6.0,Canada,1
4,PRJNA799127,44.019600,4.0,-92.481100,4.0,USA,1
...,...,...,...,...,...,...,...
52224,PRJNA237344,12.410000,2.0,-52.220000,2.0,uncalculated,2
52225,PRJNA498721,30.653210,5.0,-87.792099,6.0,USA,1
52226,PRJNA498721,30.618544,6.0,-87.784645,6.0,USA,1
52227,PRJNA498721,30.653210,5.0,-87.792099,6.0,USA,1


In [28]:
pd_output["GEO_QUAL"].value_counts()

GEO_QUAL
1    180606
2     75995
3     35226
4     21550
Name: count, dtype: int64

In [31]:
pd_output[pd_output["GEO_QUAL"] == 1]

,bioproject,acc,organism,assay_type,instrument,librarylayout,libraryselection,librarysource,geo_loc_name_country_calc,geo_loc_name_country_continent_calc,...,lat_lon_raw,latitude,longitude,latitude_precision,has_latlon,longitude_precision,rg_country_code,rg_city,rg_country,GEO_QUAL
0,PRJNA701021,SRR13668983,aquatic metagenome,AMPLICON,Sequel,SINGLE,PCR,METAGENOMIC,Canada,North America,...,55.226417 N 77.695722 W,55.226417,-77.695722,6.0,True,6.0,CA,Waskaganish,Canada,1
2,PRJNA780327,SRR16943517,aquatic metagenome,AMPLICON,MinION,SINGLE,PCR,METAGENOMIC,USA,North America,...,40.48816 N 74.42673 W,40.488160,-74.426730,5.0,True,5.0,US,Highland Park,United States,1
3,PRJNA701021,SRR13669015,aquatic metagenome,AMPLICON,Sequel,SINGLE,PCR,METAGENOMIC,Canada,North America,...,55.226528 N 77.696917 W,55.226528,-77.696917,6.0,True,6.0,CA,Waskaganish,Canada,1
4,PRJNA799127,SRR17734756,freshwater metagenome,WGS,GridION,SINGLE,RANDOM,GENOMIC,USA,North America,...,44.0196 N 92.4811 W,44.019600,-92.481100,4.0,True,4.0,US,Rochester,United States,1
5,PRJNA591360,SRR10522099,freshwater metagenome,AMPLICON,454 GS,SINGLE,PCR,METAGENOMIC,USA,North America,...,47.60583 N 86.81778 W,47.605830,-86.817780,5.0,True,5.0,US,Munising,United States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52222,PRJNA498721,SRR8127881,freshwater metagenome,AMPLICON,Illumina Genome Analyzer,PAIRED,PCR,OTHER,USA,North America,...,30.618544 N 87.784645 W,30.618544,-87.784645,6.0,True,6.0,US,Loxley,United States,1
52225,PRJNA498721,SRR8127947,freshwater metagenome,AMPLICON,Illumina Genome Analyzer,PAIRED,PCR,OTHER,USA,North America,...,30.65321 N 87.792099 W,30.653210,-87.792099,5.0,True,6.0,US,Loxley,United States,1
52226,PRJNA498721,SRR8127884,freshwater metagenome,AMPLICON,Illumina Genome Analyzer,PAIRED,PCR,OTHER,USA,North America,...,30.618544 N 87.784645 W,30.618544,-87.784645,6.0,True,6.0,US,Loxley,United States,1
52227,PRJNA498721,SRR8127887,freshwater metagenome,AMPLICON,Illumina Genome Analyzer,PAIRED,PCR,OTHER,USA,North America,...,30.65321 N 87.792099 W,30.653210,-87.792099,5.0,True,6.0,US,Loxley,United States,1
